# Speed of Light from Maxwell's Equations

This notebook contains the programmatic verification for the **Speed of Light from Maxwell's Equations** entry from the THEORIA dataset.

**Entry ID:** speed_of_light  
**Required Library:** sympy 1.13.1

## Description
In vacuum, Maxwell's equations imply that disturbances of the electromagnetic field propagate as waves whose speed is fixed by the electric permittivity and magnetic permeability of free space. Identifying this speed with the experimentally measured speed of light shows that light is an electromagnetic wave and sets the value of the universal constant `c` from purely electric and magnetic static measurements such as the force between charges or currents.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.13.1

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp
from sympy.vector import CoordSys3D, curl, divergence, gradient, laplacian

# Step 1: Start from Maxwell's equations in vacuum and define the electric field.
# We assume in vacuum:
#   curl(E) = -∂B/∂t        (Faraday's law)
#   curl(B) = μ_0 ε_0 ∂E/∂t (Ampère–Maxwell law with ρ = 0, J = 0)
# Here we represent a generic time-dependent electric field E(x, y, z, t).
mu_0, epsilon_0 = sp.symbols('mu_0 epsilon_0', positive=True, real=True)
t = sp.symbols('t')
N = CoordSys3D('N')
x, y, z = N.x, N.y, N.z

Ex = sp.Function('Ex')(x, y, z, t)
Ey = sp.Function('Ey')(x, y, z, t)
Ez = sp.Function('Ez')(x, y, z, t)
E_vec = Ex*N.i + Ey*N.j + Ez*N.k

# Step 2: Take the curl of Faraday's law:
#   curl(E) = -∂B/∂t  ⇒  curl(curl(E)) = -∂/∂t (curl(B)).
# We encode this at the level of scalar placeholders:
curlE_s, B_t_s = sp.symbols('curlE_s B_t_s', real=True)
curlcurlE_s, curlB_t_s = sp.symbols('curlcurlE_s curlB_t_s', real=True)

eq_faraday = sp.Eq(curlE_s, -B_t_s)          # curl(E) = -∂B/∂t
eq2_form  = sp.Eq(curlcurlE_s, -curlB_t_s)   # curl(curl(E)) = -∂/∂t(curl(B))

# Step 3: Use Ampère–Maxwell law in vacuum:
#   curl(B) = μ_0 ε_0 ∂E/∂t  ⇒  ∂/∂t(curl(B)) = μ_0 ε_0 ∂²E/∂t².
curlB_s, E_t, E_tt = sp.symbols('curlB_s E_t E_tt', real=True)
eq_ampere = sp.Eq(curlB_s, mu_0*epsilon_0*E_t)
eq3_time  = sp.Eq(curlB_t_s, mu_0*epsilon_0*E_tt)

# Combine Step 2 and Step 3 to obtain:
#   curl(curl(E)) = - μ_0 ε_0 ∂²E/∂t².
eq2_3_derived  = sp.Eq(eq2_form.lhs, eq2_form.rhs.subs(curlB_t_s, eq3_time.rhs))
eq2_3_expected = sp.Eq(curlcurlE_s, -mu_0*epsilon_0*E_tt)
assert sp.simplify(eq2_3_derived.rhs - eq2_3_expected.rhs) == 0

# Step 4: Apply the vector identity
#   curl(curl(E)) = grad(div(E)) - ∇²E
# and verify it explicitly with SymPy for a general E(x, y, z, t).
curlE_vec       = curl(E_vec)
curlcurlE_vec   = curl(curlE_vec)
divE            = divergence(E_vec)
grad_divE_vec   = gradient(divE)
lapE_vec        = laplacian(E_vec)

identity_vec = sp.simplify(curlcurlE_vec - (grad_divE_vec - lapE_vec))
# SymPy represents the zero vector as a special VectorZero object; we check that
# its magnitude is zero.
if hasattr(identity_vec, "magnitude"):
    assert identity_vec.magnitude() == 0
else:
    assert identity_vec == 0

# Step 5: In vacuum, Gauss's law gives div(E) = 0 ⇒ grad(div(E)) = 0.
# In scalar placeholder form, write:
lapE_scalar, graddivE_s = sp.symbols('lapE_scalar graddivE_s', real=True)
eq4_scalar = sp.Eq(curlcurlE_s, graddivE_s - lapE_scalar)
eq5        = sp.Eq(graddivE_s, 0)

# Substitute eq5 into eq4_scalar to obtain curl(curl(E)) = -∇²E.
eq4_reduced = sp.Eq(eq4_scalar.lhs, eq4_scalar.rhs.subs(graddivE_s, 0))
assert sp.simplify(eq4_reduced.rhs + lapE_scalar) == 0  # gives curlcurlE = -lapE

# Step 6: Equate the two expressions for curl(curl(E)):
#   from Step 3: curlcurlE = - μ_0 ε_0 E_tt
#   from Step 5: curlcurlE = -lapE
# and solve for lapE to obtain the wave equation
#   ∇²E = μ_0 ε_0 ∂²E/∂t².
wave_eq_rhs = sp.simplify(sp.solve(
    sp.Eq(eq2_3_expected.rhs, eq4_reduced.rhs),
    lapE_scalar
)[0])
assert sp.simplify(wave_eq_rhs - mu_0*epsilon_0*E_tt) == 0

# Step 7: Recognize the standard wave equation
#   ∇²F = (1/v²) ∂²F/∂t²
# so that matching coefficients gives 1/v² = μ_0 ε_0.
v = sp.symbols('v', positive=True, real=True)
eq7 = sp.Eq(1/v**2, mu_0*epsilon_0)
v_solution = sp.solve(eq7, v)[0]
assert sp.simplify(v_solution - 1/sp.sqrt(mu_0*epsilon_0)) == 0

# Step 8: Define c as the propagation speed of electromagnetic waves in vacuum,
#   c = v = 1/sqrt(μ_0 ε_0),
# and check that this is equivalent to 1/c² = μ_0 ε_0.
c = sp.symbols('c', positive=True, real=True)
eq_c_def = sp.Eq(c, v_solution)
eq_c_inv = sp.Eq(1/c**2, mu_0*epsilon_0)
lhs_sub = eq_c_inv.lhs.subs(c, eq_c_def.rhs)
assert sp.simplify(lhs_sub - eq_c_inv.rhs) == 0

print("Programmatic verification of Steps 1–8 for speed_of_light entry passed.")


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=speed_of_light.json](https://theoria-dataset.org/entries.html?entry=speed_of_light.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0